<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bo-YAN---HW-1-Question-2" data-toc-modified-id="Bo-YAN---HW-1-Question-2-1">Bo YAN - HW 1 Question 2</a></span><ul class="toc-item"><li><span><a href="#Using-organization-names-from-Patentholders.csv-that-have-the-string-&quot;Inc&quot;-(i.e.,-incorporated)-find-similar-values-in-the-orgNames.csv-data-set.-Your-output-should-be-a-table-where-the-first-column-is-a-name-from-Patentholders.csv-and-the-second-column-is-a-list-of-matching-values-from-orgNames.csv." data-toc-modified-id="Using-organization-names-from-Patentholders.csv-that-have-the-string-&quot;Inc&quot;-(i.e.,-incorporated)-find-similar-values-in-the-orgNames.csv-data-set.-Your-output-should-be-a-table-where-the-first-column-is-a-name-from-Patentholders.csv-and-the-second-column-is-a-list-of-matching-values-from-orgNames.csv.-1.1">Using organization names from Patentholders.csv that have the string "Inc" (i.e., incorporated) find similar values in the orgNames.csv data set. Your output should be a table where the first column is a name from Patentholders.csv and the second column is a list of matching values from orgNames.csv.</a></span></li></ul></li><li><span><a href="#1.-Load-data" data-toc-modified-id="1.-Load-data-2">1. Load data</a></span></li><li><span><a href="#2.-Create-functions-and-declarations" data-toc-modified-id="2.-Create-functions-and-declarations-3">2. Create functions and declarations</a></span><ul class="toc-item"><li><span><a href="#2.1---Functions" data-toc-modified-id="2.1---Functions-3.1">2.1 - Functions</a></span></li><li><span><a href="#2.2---Declarations" data-toc-modified-id="2.2---Declarations-3.2">2.2 - Declarations</a></span></li></ul></li><li><span><a href="#3.-Filter-data" data-toc-modified-id="3.-Filter-data-4">3. Filter data</a></span></li><li><span><a href="#4.-Apply-pruning-methods-and-find-the-matching-values" data-toc-modified-id="4.-Apply-pruning-methods-and-find-the-matching-values-5">4. Apply pruning methods and find the matching values</a></span></li><li><span><a href="#5.-Concate-dataframes" data-toc-modified-id="5.-Concate-dataframes-6">5. Concate dataframes</a></span></li><li><span><a href="#6.-Output-table" data-toc-modified-id="6.-Output-table-7">6. Output table</a></span></li><li><span><a href="#7.-Time-complexity" data-toc-modified-id="7.-Time-complexity-8">7. Time complexity</a></span><ul class="toc-item"><li><span><a href="#The-time-complexity-after-filtering-and-pruning-is-around-5-minutes.-If-we-loose-the-constraints,-it-may-take-more-time." data-toc-modified-id="The-time-complexity-after-filtering-and-pruning-is-around-5-minutes.-If-we-loose-the-constraints,-it-may-take-more-time.-8.1">The time complexity after filtering and pruning is around 5 minutes. If we loose the constraints, it may take more time.</a></span></li></ul></li></ul></div>

# Bo YAN - HW 1 Question 2

## Using organization names from Patentholders.csv that have the string "Inc" (i.e., incorporated) find similar values in the orgNames.csv data set. Your output should be a table where the first column is a name from Patentholders.csv and the second column is a list of matching values from orgNames.csv.

In [1]:
import py_stringmatching as sm 
import pandas as pd
import numpy as np 
from sklearn.metrics import precision_score, recall_score
from collections import Counter
import re
import warnings
warnings.filterwarnings("ignore")

# 1. Load data
    ▪ Load patent_holders data
    ▪ Load org_names data

In [2]:
patent_holders_data = pd.read_csv('Patentholders.csv', header = None, names = ['patent_holders'])
patent_holders_data.drop_duplicates(subset = 'patent_holders', inplace = True)
print(patent_holders_data.shape)
patent_holders_data.head(10)

(7620, 1)


,patent_holders
0,Signify Holding BV
1,ThyssenKrupp Airport Systems SA
2,Eurenco
3,EI du Pont de Nemours and Co
4,TRIATEX INTERNATIONAL AG
5,"THYSSENKRUPP TAILORED BLANKS GMBH, 47139 DUISB..."
6,Howe Wayne R
7,Carbone Lorraine Composants
8,Google LLC
9,MACOM Technology Solutions Holdings Inc


In [3]:
org_names_data = pd.read_csv('orgNames.csv', header = None, names = ['org_names'])
org_names_data.drop_duplicates(subset = 'org_names', inplace = True)
org_names_data.dropna(subset=['org_names'])
print(org_names_data.shape)
org_names_data.head(10)

(443539, 1)


,org_names
0,00 jackson state
1,0150
2,01 trump
3,02
4,02512
5,0263
6,0270
7,02 father of merced
8,02 meet
9,03


# 2. Create functions and declarations

## 2.1 - Functions

In [4]:
def filter_by_string(data, string, column_name):
    data = data[data[column_name].str.contains(string, case = False, na = False)]
    data = data[data[column_name].str.endswith(string)]
    data.reset_index(drop = True, inplace= True)
    return data

In [5]:
# Common tokens for Jaccard token based similarity: T = t/(1 + t) * (|r| + |s|)
def pruning_by_word_count(source_data_list, target_data_list, source_data_index, threshold):    
    target_data_list_after_pruning_by_word_count = []
    source_data_list_word_count = len(str(source_data_list[source_data_index]).split())
    
    for item in target_data_list:              
        target_data_list_word_count = len(str(item).split())
        
        # common tokens we need
        tokens_number_need = threshold/(1 + threshold) * (source_data_list_word_count + target_data_list_word_count)
    
        # find common tokens number
        common_tokens_number = len(set(str(source_data_list[source_data_index]).lower().split()) & set(str(item).lower().split()))
        
        # if common token number is over the needed, then add this to our list
        if common_tokens_number >= tokens_number_need:
            target_data_list_after_pruning_by_word_count.append(item)    
    
    return target_data_list_after_pruning_by_word_count

In [6]:
def pruning_by_word_size(source_data_list, target_data_list, source_data_index, word_size_width):   
    word_size = len(source_data_list[source_data_index].split())    
    target_data_list_after_pruning_by_word_size = []
    for item in target_data_list:
        size_constraint = (word_size - word_size_width <= len(str(item).split()) <= word_size + word_size_width)
        # if we want to use a strict size constraint match, we can apply the line below.
        # size_constraint = len(str(item).split())       
        if size_constraint:
            target_data_list_after_pruning_by_word_size.append(item)
    return target_data_list_after_pruning_by_word_size

In [7]:
def pruning_by_prefix_filtering(source_data_list, target_data_list_after_pruning_by_word_size, source_data_index, start_letter_index, end_letter_index):
    final_target_data_list = []
    for item in target_data_list_after_pruning_by_word_size:
        start_letters = source_data_list[source_data_index].lower()[start_letter_index:end_letter_index]
        prefix_filtering = str(item)[start_letter_index:end_letter_index]        
        if prefix_filtering == start_letters:
            final_target_data_list.append(item)
    return final_target_data_list

In [8]:
def find_matching_values(source_data_list, final_target_data_list, source_data_index, threshold_value, similarity_measure, word_token):
    result = []
    for item in final_target_data_list:
        score = similarity_measure.get_sim_score(word_token.tokenize(source_data_list[source_data_index].lower()), word_token.tokenize(item.lower()))
        if score > threshold_value:
            result.append(item)
    return result

## 2.2 - Declarations

In [9]:
# create a whitespace tokenizer
ws_tok = sm.WhitespaceTokenizer()

# create a qgram tokenizer using q = 3
qg3_tok = sm.QgramTokenizer(qval = 3)

# create a Jaccard similarity measure object
jac = sm.Jaccard()

# 3. Filter data

In [10]:
patent_holders_data = filter_by_string(patent_holders_data, string = 'Inc', column_name = 'patent_holders')
print(patent_holders_data.shape)
patent_holders_data.head(10)

(2063, 1)


,patent_holders
0,MACOM Technology Solutions Holdings Inc
1,CONFIDENT TECHNOLOGIES Inc
2,Bitsysoft Technology Inc
3,Specialty Manufacturing Inc
4,Bell Helicopter Michigan Inc
5,Autodesk Inc
6,Alert Logic Inc
7,BOOZ-ALLEN HAMILTON Inc
8,AFX Tech Group International Inc
9,CTB Inc


In [11]:
org_names_data = filter_by_string(org_names_data, string = 'inc', column_name = 'org_names')
print(org_names_data.shape)
org_names_data.head(10)

(42275, 1)


,org_names
0,03sl industries inc
1,101 outdoors inc
2,10-20 services inc
3,10-31 inc
4,10-8 tactical inc
5,1105 media inc
6,1150018 ontario inc
7,"11, inc"
8,"1220 exhibits, inc"
9,"132 group, inc"


# 4. Apply pruning methods and find the matching values

In [12]:
final_result = []
patent_holders_data_list = patent_holders_data['patent_holders'].tolist()
org_names_data_list = org_names_data['org_names'].tolist()

for index in range(len(patent_holders_data_list)):
    
    org_names_data_list = pruning_by_word_count(patent_holders_data_list, org_names_data_list, source_data_index = index, threshold = 0.1)
    
    # if we would like to strict/loose the word size, simply change the parameter word_size_width.
    target_data_list_after_pruning_by_word_size = pruning_by_word_size(patent_holders_data_list, org_names_data_list, source_data_index = index, word_size_width = 1)
    
    # if you would like to strict/loose the prefix filtering, simply change the parameters start_letter_index and end_letter_index.
    final_target_data_list = pruning_by_prefix_filtering(patent_holders_data_list, target_data_list_after_pruning_by_word_size, source_data_index = index, start_letter_index = 0, end_letter_index = 2)
    
    # if you would like to change the threshould, similarity measure and tokenize method, simply change the parameter threshold_value, similarity_measure, and word_token and declaring them before consume them.
    matching_value_list = find_matching_values(patent_holders_data_list, final_target_data_list, source_data_index = index, threshold_value = 0.5, similarity_measure = jac, word_token = ws_tok)
    
    # join matching values for each city
    final_result.append(' || '.join(matching_value_list))

In [13]:
final_result[0:10]

['macom technology solutions holdings inc || macom technology solutions inc',
 'confident technologies inc',
 '',
 'specialty manufacturing inc',
 'bell helicopter michigan inc || bell helicopter textron inc || bell helicopter textron, inc',
 'autodesk inc',
 'alert logic inc',
 'booz-allen and hamilton inc || booz-allen hamilton inc',
 'afx tech group international inc',
 'ctb inc']

# 5. Concate dataframes

In [14]:
df1 = pd.DataFrame(patent_holders_data_list)
df1.columns = ['organization_name']
df1.head(10)

,organization_name
0,MACOM Technology Solutions Holdings Inc
1,CONFIDENT TECHNOLOGIES Inc
2,Bitsysoft Technology Inc
3,Specialty Manufacturing Inc
4,Bell Helicopter Michigan Inc
5,Autodesk Inc
6,Alert Logic Inc
7,BOOZ-ALLEN HAMILTON Inc
8,AFX Tech Group International Inc
9,CTB Inc


In [15]:
df2 = pd.DataFrame(final_result)
df2.columns = ['org_name']
df2.head(10)

,org_name
0,macom technology solutions holdings inc || mac...
1,confident technologies inc
2,
3,specialty manufacturing inc
4,bell helicopter michigan inc || bell helicopte...
5,autodesk inc
6,alert logic inc
7,booz-allen and hamilton inc || booz-allen hami...
8,afx tech group international inc
9,ctb inc


In [16]:
f_column = df2['org_name']
df1 = pd.concat([df1, f_column], axis = 1)
df1.head(10)

,organization_name,org_name
0,MACOM Technology Solutions Holdings Inc,macom technology solutions holdings inc || mac...
1,CONFIDENT TECHNOLOGIES Inc,confident technologies inc
2,Bitsysoft Technology Inc,
3,Specialty Manufacturing Inc,specialty manufacturing inc
4,Bell Helicopter Michigan Inc,bell helicopter michigan inc || bell helicopte...
5,Autodesk Inc,autodesk inc
6,Alert Logic Inc,alert logic inc
7,BOOZ-ALLEN HAMILTON Inc,booz-allen and hamilton inc || booz-allen hami...
8,AFX Tech Group International Inc,afx tech group international inc
9,CTB Inc,ctb inc


# 6. Output table

In [17]:
df1.head(10)

,organization_name,org_name
0,MACOM Technology Solutions Holdings Inc,macom technology solutions holdings inc || mac...
1,CONFIDENT TECHNOLOGIES Inc,confident technologies inc
2,Bitsysoft Technology Inc,
3,Specialty Manufacturing Inc,specialty manufacturing inc
4,Bell Helicopter Michigan Inc,bell helicopter michigan inc || bell helicopte...
5,Autodesk Inc,autodesk inc
6,Alert Logic Inc,alert logic inc
7,BOOZ-ALLEN HAMILTON Inc,booz-allen and hamilton inc || booz-allen hami...
8,AFX Tech Group International Inc,afx tech group international inc
9,CTB Inc,ctb inc


# 7. Time complexity

## The time complexity after filtering and pruning is around 5 minutes. If we loose the constraints, it may take more time.